In [2]:
#뉴스데이터 불러오기
import pickle
import pandas as pd

with open('naver_data.pkl', 'rb') as f:
    data1 = pickle.load(f)
#댓글 데이터 불러오기
data2 = pd.read_csv('naver_comments_all.csv')
data2.head(2)

,press,writer,title,reg_date,mod_date,sub_cat,sub_catid,url,platform,contents,comments,sticker,comment,user_id,user_name,date_upload,date_fix,good_cnt,bad_cnt
0,서울경제,"유주희 기자(ginger@sedaily.com),팀지구용 기자(use4us@seda...","아이폰 배터리, 직접 교체해 봤습니다[지구용]",2023-07-02 08:51:01,NaN,모바일,731,https://n.news.naver.com/mnews/article/011/000...,Naver,"['\n\t\t\t', '[서울경제] ', '아직 멀쩡한 폰을 놔두고 새 폰을 산 ...","['796344990274945052', 'M자 탈모폰이 뭔 그리 좋다고.\n국산품...","{'touched': 0, 'warm': 0, 'analytical': 0, 'li...",M자 탈모폰이 뭔 그리 좋다고.\n국산품 씁시다,tvRC,cycy****,2023-07-03T08:01:06+0900,2023-07-03T08:01:06+0900,0,1
1,서울경제,"유주희 기자(ginger@sedaily.com),팀지구용 기자(use4us@seda...","아이폰 배터리, 직접 교체해 봤습니다[지구용]",2023-07-02 08:51:01,NaN,모바일,731,https://n.news.naver.com/mnews/article/011/000...,Naver,"['\n\t\t\t', '[서울경제] ', '아직 멀쩡한 폰을 놔두고 새 폰을 산 ...","['796342807861133371', '뜯는순간 물샌다는건 잊지 마세요~', '...","{'touched': 0, 'warm': 0, 'analytical': 0, 'li...",뜯는순간 물샌다는건 잊지 마세요~,wyME,seaj****,2023-07-03T07:27:14+0900,2023-07-03T07:27:14+0900,0,0


In [3]:
# 데이터 그룹화
data3 = data2.groupby('url')['user_id'].unique()
data3.head(1)

url
https://n.news.naver.com/mnews/article/001/0013997566?sid=105    [55HyA, 1gMlZ, CioL]
Name: user_id, dtype: object

In [4]:
#벡터화
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

# URL별 유니크한 user_id들을 공백으로 연결
data3_str = data3.apply(lambda x: ' '.join(x))

# Counter Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data3_str)

# LSA (Latent Semantic Analysis)
svd = TruncatedSVD(n_components=10)  # n_components는 잠재 변수의 수, 적절한 값을 선택해야 합니다.
lsa_result = svd.fit_transform(X)

print(lsa_result)


[[ 2.90461366e-04  5.86859105e-04 -6.65942641e-05 ... -3.22974569e-03
   3.56915086e-03  4.18518852e-04]
 [ 1.12477258e-02  4.65996027e-04 -2.15638517e-03 ... -5.14751656e-03
  -4.94099014e-03  1.71738097e-03]
 [ 1.38295473e-02  1.69957126e-02  3.67511073e-04 ... -2.07952506e-02
  -8.04343559e-03 -7.09646380e-03]
 ...
 [ 4.43751144e-02  4.74671312e-02 -2.40914209e-02 ... -2.06257138e-03
   2.07743410e-02 -2.64200826e-04]
 [ 5.31185726e-17 -6.69516356e-17  8.85741356e-15 ... -3.79319531e-15
  -6.58265387e-14  1.08678926e-14]
 [ 5.49916447e-02  3.16740706e-02  4.60878634e-02 ... -1.01251596e-01
  -2.75342425e-01  6.23679885e-01]]


In [6]:
#코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity

# LSA 결과에서 코사인 유사도 계산
cosine_sim = cosine_similarity(lsa_result)

# URL과 인덱스 매핑 딕셔너리 생성
indices = pd.Series(range(len(data3_str.index)), index=data3_str.index)



In [10]:
# 기사링크 기반 추천시스템
def recommend(url, cosine_sim=cosine_sim):
    # 선택한 URL의 인덱스 가져오기
    idx = indices[url]

    # 모든 URL에 대해 해당 URL과의 유사도 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 URL들 정렬하기
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 URL 가져오기 (자신 제외)
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 URL 인덱스 가져오기
    url_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 URL 반환하기
    return data3_str.index[url_indices]

In [7]:
#코사인유사도
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# 데이터 전처리: 각 user_id별로 방문한 url들을 공백으로 연결
df2_str_user = data2.groupby('user_id')['url'].apply(lambda x: ' '.join(x.values))
# Counter Vectorization
vectorizer = CountVectorizer()
X_user = vectorizer.fit_transform(df2_str_user)
# LSA (Latent Semantic Analysis)
svd = TruncatedSVD(n_components=10)  # n_components는 잠재 변수의 수, 적절한 값을 선택해야 합니다.
lsa_result_user = svd.fit_transform(X_user)
# User ID와 인덱스 매핑 딕셔너리 생성
indices_user = pd.Series(range(len(df2_str_user.index)), index=df2_str_user.index)


In [9]:
#유저기반 추천시스템 
def recommend_urls_for_users(user_id):
    # 선택된 user의 인덱스 가져오기
    idx = indices_user[user_id]
    # 모든 URL에 대해 해당 user와의 유사도 가져오기
    sim_scores_url = cosine_similarity(lsa_result_user[idx].reshape(1,-1), lsa_result)
    # 유사도에 따라 url 정렬하기
    sim_scores_url_sorted_indices = np.argsort(sim_scores_url.flatten())
    # 가장 유사한 10개의 url 가져오기
    top_5_similar_urls_indices = sim_scores_url_sorted_indices[:5]
    return data3_str.index[top_5_similar_urls_indices]


Index(['https://n.news.naver.com/mnews/article/081/0003379356?sid=105',
       'https://n.news.naver.com/mnews/article/030/0003121094?sid=105',
       'https://n.news.naver.com/mnews/article/421/0006925346?sid=105',
       'https://n.news.naver.com/mnews/article/079/0003797876?sid=105',
       'https://n.news.naver.com/mnews/article/001/0014072421?sid=105'],
      dtype='object', name='url')

In [14]:
#출력확인
recommend_urls_for_users('tvRC')

Index(['https://n.news.naver.com/mnews/article/421/0006896433?sid=105',
       'https://n.news.naver.com/mnews/article/421/0007031902?sid=105',
       'https://n.news.naver.com/mnews/article/277/0005278876?sid=105',
       'https://n.news.naver.com/mnews/article/277/0005278822?sid=105',
       'https://n.news.naver.com/mnews/article/092/0002302921?sid=105'],
      dtype='object', name='url')

In [26]:
#딕셔너리 생성
dict1 = {name: id for id, name in zip(data2.user_id, data2.user_name)}
dict1.keys()

dict_keys(['cycy****', 'seaj****', 'lks0****', 'dudx****', 'rnjs****', 'last****', 'mina****', 'cksd****', 'jack****', 'kore****', 'dsmr****', 'khb7****', 'youg****', 'spee****', 'laal****', 'mini****', 'olo6****', 'sung****', 'vivi****', 'cavi****', 'shel****', 'choo****', 'jya2****', 'ceo2****', 'espo****', 'yujc****', 'park****', 'skgm****', 'kaze****', 'ksan****', 'qkrg****', 'woqk****', 'lky2****', 'osc0****', 'do11****', 'thir****', 'shmk****', 'vkfk****', 'kool****', 'ramt****', 'youn****', 'eija****', 'suho****', 'aoom****', 'habm****', 'jck2****', 'kims****', 'litp****', 'baro****', 'ja_s****', 'fltm****', 'love****', 'kill****', 'rlaa****', 'gunj****', 'cjkn****', '1106****', 'hdec****', 'nam1****', 'wdhl****', 'devi****', 'kidc****', 'land****', 'ceda****', 'insu****', 'ji7e****', 'rkak****', 'dood****', 'kijo****', 'lees****', 'mush****', 'dunk****', 'bs******', 'eowl****', 'jams****', 'aatn****', '89gm****', 'batt****', 'bush****', 'kimh****', 'dldy****', 'hete****', 'pjyn

In [24]:
#아이디를 입력받아 뉴스 추천5개
a = input("아이디를 입력하세요 : ")
recommend_urls_for_users(dict1[a[:4] + '****'])

Index(['https://n.news.naver.com/mnews/article/421/0006896433?sid=105',
       'https://n.news.naver.com/mnews/article/421/0007031902?sid=105',
       'https://n.news.naver.com/mnews/article/277/0005278876?sid=105',
       'https://n.news.naver.com/mnews/article/277/0005278822?sid=105',
       'https://n.news.naver.com/mnews/article/092/0002302921?sid=105'],
      dtype='object', name='url')

'tvRC'